In [39]:
import pandas as pd

In [40]:
df = pd.read_csv("Calgary_Transit_Ridership_20251030.csv")


In [41]:
df = df.drop_duplicates()

In [42]:
ridership_cols = ['Year', 'Month', 'Ridership', 'Youth Ridership', 'Adult Ridership', 'U-Pass Ridership' ]
df = df[ridership_cols]

In [45]:
df = df.dropna()
pd.set_option('display.max_rows', None)

In [ ]:
df = df.reset_index(drop = True)
df

,Year,Month,Ridership,Youth Ridership,Adult Ridership,U-Pass Ridership
0,2025,Jan,"8,147,400","1,068,110","4,554,859","1,032,000"
1,2024,Dec,"7,674,900","1,073,674","4,244,065","698,000"
2,2024,Nov,"8,548,900","1,269,241","4,577,247","985,500"
3,2024,Oct,"10,032,900","1,493,054","5,408,299","1,196,900"
4,2024,Sep,"9,364,500","1,469,657","5,005,432","1,148,800"
5,2024,Aug,"7,740,400","587,170","5,178,520","306,300"
6,2024,Jul,"7,806,200","714,227","4,942,520","371,600"
7,2024,Jun,"7,901,100","1,031,024","4,780,933","386,700"
8,2024,May,"8,677,500","1,340,179","5,253,278","416,900"
9,2024,Apr,"7,859,800","1,175,821","4,322,748","818,600"
